## Import Dependencies

In [ ]:
# misc
import math
import time
from pathlib import Path
from fractions import Fraction

# scientific
import numpy as np
import beatbrain
from beatbrain import utils

# visualization
from IPython import display
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Tensorflow
import tensorflow as tf

from tensorflow.keras import Model, Sequential, Input, optimizers
from tensorflow.keras import layers, losses
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Lambda,
    Reshape,
    Flatten,
    Layer,
    Add,
    Multiply,
    Bidirectional,
    BatchNormalization,
    ReLU,
    Activation,
    Permute,
    RepeatVector,
)
from tensorflow.keras.callbacks import (
    Callback,
    TensorBoard,
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TerminateOnNaN,
    CSVLogger,
    LambdaCallback,
)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
sns.set()
sns.set_style("white")
%matplotlib inline

## Load Datasets

In [ ]:
# Input
DATA_ROOT = Path("../data/fma/image")
IMAGE_DIMS = [512, 640, 1]
BATCH_SIZE = 4

In [ ]:
train_dataset = utils.load_dataset(
    DATA_ROOT / "train", batch_size=BATCH_SIZE, parallel=False,
)
val_dataset = utils.load_dataset(
    DATA_ROOT / "val", batch_size=BATCH_SIZE, parallel=False,
)
test_dataset = utils.load_dataset(
    DATA_ROOT / "test", batch_size=1, parallel=False, shuffle_buffer=0,
)

## Define Model Architecture

In [ ]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2.0 * np.pi)
    return tf.reduce_sum(
        -0.5 * ((sample - mean) ** 2.0 * tf.exp(-logvar) + logvar + log2pi), axis=raxis
    )


@tf.function
def sample(latent_dim, decoder, eps=None):
    if eps is None:
        eps = tf.random.normal(shape=(100, latent_dim))
    return decode(decoder, eps, apply_sigmoid=True)


def encode(encoder, x):
    inference = encoder(x)
    mean, logvar = tf.split(inference, num_or_size_splits=2, axis=1)
    return mean, logvar


def reparameterize(mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * 0.5) + mean


def decode(decoder, z, apply_sigmoid=False):
    logits = decoder(z)
    if apply_sigmoid:
        probs = tf.sigmoid(logits)
        return probs
    return logits


def tanh_to_sigmoid(x):
    """
    Convert a tensor from `tanh` activation to `sigmoid` activation
    """
    return ((x / 2) + 1) / 2

In [ ]:
# Custom layers
class ReconstructionLoss(Layer):
    # TODO: subclass `Loss` instead
    def __init__(self, mean=True):
        self._mean = mean
        super().__init__()
        
    def _mse(self, x):
        return tf.reduce_sum(tf.losses.mse(x[0], x[1]), axis=[1, 2])

    def build(self, input_shape):
        self._model = Sequential([
            Lambda(self._mse)
        ])
        if self._mean:
            self._model.add(Lambda(tf.reduce_mean))
        super().build(input_shape)

    def call(self, x):
        return self._model(x)

    def compute_output_shape(self, input_shape):
        return self._model.compute_output_shape(input_shape)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "mean": self._mean,
        })
        return config


class KLLoss(Layer):
    # TODO: subclass `Loss` instead
    def __init__(self, mean=True):
        self._mean = mean
        super().__init__()

    def _log_normal_pdf(self, sample, mean, logvar, raxis=1):
        log2pi = tf.math.log(2.0 * np.pi)
        return tf.reduce_sum(
            -0.5 * ((sample - mean) ** 2.0 * tf.exp(-logvar) + logvar + log2pi), axis=raxis
        )

    def _kld(self, x):
        return self._log_normal_pdf(x[0], x[1], x[2]) - self._log_normal_pdf(x[0], 0., 0.)

    def build(self, input_shape):
        self._model = Sequential([
            Lambda(self._kld)
        ])
        if self._mean:
            self._model.add(Lambda(tf.reduce_mean))
        super().build(input_shape)

    def call(self, x):
        z, z_mean, z_log_var = x
        return self._model([z, z_mean, z_log_var])

    def compute_output_shape(self, input_shape):
        return self._model.compute_output_shape(input_shape)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "mean": self._mean,
        })
        return config


class Reparameterize(Layer):
    def build(self, input_shape):
        print("Reparam Layer input shape:", input_shape)
        inputs = Input(shape=[x[1:] for x in input_shape])
        print("Reparam input shape:", inputs.shape)
        # TODO: Get rid of lambda expressions inside Lambda layers
        epsilon = Lambda(lambda x: tf.keras.backend.random_normal(shape=tf.shape(x[0])))(inputs)
        print("Epsilon shape:", epsilon.shape)
        mean = Lambda(lambda x: x[0])(inputs)
        print("Mean shape:", mean.shape)
        var = Lambda(lambda x: tf.exp(x[1] * 0.5))(inputs)
        print("Var shape:", var.shape)
        reparam = Multiply()([epsilon, var])
        print("Mul shape:", reparam.shape)
        reparam = Add()([reparam, mean])
        print("Add shape:", reparam.shape)
        self._model = Model(inputs=inputs, outputs=reparam)
        print("Reparam output shape:", self._model.output_shape)
        super().build(input_shape)

    def call(self, x):
        z_mean, z_log_var = x
        output = self._model([z_mean, z_log_var])
        print("Call time shape:", output.shape)
        return output

    def compute_output_shape(self, input_shape):
        return self._model.compute_output_shape(input_shape)


def build_encoder(input_shape, latent_dim, batch_size, repeat, use_inception):
    def reparam(args):
        z_mean, z_log_var = args
        dim = tf.keras.backend.int_shape(z_mean)[1]
        eps = tf.keras.backend.random_normal(shape=tf.shape(z_mean))
        return eps * tf.exp(z_log_var * 0.5) + z_mean

    encoder_input = Input(
        shape=input_shape,
        name="encoder_input"
    )
    e = Reshape(input_shape[:-1])(encoder_input)  # Equivalent to squeezing channels axis
    e = Permute((2, 1))(e)  # Convert to time-major format [batch, time, freq]
    e = Bidirectional(LSTM(latent_dim))(e)
    z_mean = Dense(latent_dim, name="z_mean")(e)
    z_log_var = Dense(latent_dim, name="z_log_var")(e)
    z = Lambda(reparam, output_shape=(latent_dim,), name="z")([z_mean, z_log_var])
    encoder = Model(inputs=encoder_input, outputs=[z_mean, z_log_var, z], name="encoder")
    return encoder_input, encoder


def build_decoder(latent_dim, output_shape, batch_size, repeat, use_inception):
    decoder_input = Input(
        shape=latent_dim,
        name="decoder_input"
    )
    d = Dense(latent_dim)(decoder_input)
    d = RepeatVector(output_shape[1])(d)
    d = Bidirectional(LSTM(output_shape[0], return_sequences=True))(d)
    d = Lambda(tanh_to_sigmoid)(d)
    d = Permute((2, 1))(d)
    d = Reshape(output_shape)(d)
#     d = Activation(tf.nn.sigmoid)(d)
    decoder = Model(inputs=decoder_input, outputs=d, name="decoder")
    return decoder_input, decoder

In [ ]:
def build_cvae(
    latent_dim,
    input_shape,
    repeat=1,
    use_inception=True,
    batch_size=1,
    learning_rate=1e-4,
):
    encoder_input, encoder = build_encoder(input_shape, latent_dim, batch_size, repeat, use_inception)
    decoder_input, decoder = build_decoder(latent_dim, input_shape, batch_size, repeat, use_inception)
    z_mean, z_log_var, z = encoder(encoder_input)
    decoder_output = decoder(z)
    model = Model(encoder_input, decoder_output, name="vae")

    print(f"Encoder input: {encoder_input.shape}")
    print(f"Decoder output: {decoder_output.shape}")
    encoder_input.shape.assert_is_compatible_with(decoder_output.shape)
#     assert encoder_input.shape.as_list() == decoder_output.shape.as_list()

    reconstruction_loss = ReconstructionLoss(mean=True)([encoder_input, decoder_output])
#     reconstruction_loss = tf.losses.mse(encoder_input, decoder_output)
#     reconstruction_loss = tf.reduce_sum(reconstruction_loss, axis=[1, 2])
    kl_loss = KLLoss(mean=True)([z, z_mean, z_log_var])
#     logpz = log_normal_pdf(z, 0.0, 0.0)
#     logqz_x = log_normal_pdf(z, z_mean, z_log_var)
#     kl_loss = logqz_x - logpz
    vae_loss = reconstruction_loss + kl_loss
    model.add_loss(vae_loss)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
    # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=lambda yt, yp: vae_loss)

    model.add_metric(reconstruction_loss, aggregation="mean", name="reconstruction_loss")
    model.add_metric(kl_loss, aggregation="mean", name="kl_loss")
    return model, encoder, decoder

## Define Training Callbacks

In [ ]:
class VisualizeCallback(Callback):
    def __init__(
        self,
        log_dir,
        latent_dim,
        validation_data,
        n_examples=4,
        random_vectors=None,
        heatmap=True,
        distribution=False,
        frequency="epoch",
        verbose=False,
    ):
        self.log_dir = Path(log_dir)
        self.latent_dim = latent_dim
        self.n_examples = n_examples
        self.cmap = "magma" if heatmap else "Greys"
        self.frequency = frequency
        self.verbose = verbose
        self.total_batch = 0
        self.distribution = distribution
        self.random_vectors = random_vectors or tf.random.normal(
            shape=[n_examples, latent_dim]
        )
        self.fig = plt.figure()
        self.samples = list(validation_data.unbatch().take(self.n_examples))

        self.recon_raw = self.log_dir / "raw" / "reconstructed"
        self.recon_png = self.log_dir / "png" / "reconstructed"
        self.gen_raw = self.log_dir / "raw" / "generated"
        self.gen_png = self.log_dir / "png" / "generated"

    def on_train_begin(self, logs=None):
        self.recon_raw.mkdir(exist_ok=True, parents=True)
        self.recon_png.mkdir(exist_ok=True, parents=True)
        self.gen_raw.mkdir(exist_ok=True, parents=True)
        self.gen_png.mkdir(exist_ok=True, parents=True)

    def _visualize_reconstruction(self, batch=None, epoch=None):
        assert (batch is not None) or (epoch is not None)
        fig = plt.figure(self.fig.number)
        fig.set_size_inches(10, 4)
        for i, sample in enumerate(self.samples):
            fig.add_subplot(121)
            sample = sample[None, :]
            beatbrain.display.show_spec(
                utils.denormalize_spectrogram(sample[0, ..., 0].numpy()),
                title="Original",
                cmap=self.cmap,
            )
            fig.add_subplot(122)
            reconstructed = self.model(sample)
            beatbrain.display.show_spec(
                utils.denormalize_spectrogram(reconstructed[0, ..., 0].numpy()),
                title="Reconstructed",
                cmap=self.cmap,
            )
            fig.tight_layout()
            title = f"recon_{i + 1}@{'epoch' if epoch else 'batch'}_{epoch or batch:06}"
            fig.suptitle(title)
            fig.savefig(self.recon_png / f"{title}.jpg")
            utils.save_image(
                reconstructed[0, ..., 0].numpy(), self.recon_raw / f"{title}.exr",
            )
            fig.clear()

        if self.distribution:
            fig = plt.figure(self.fig.number)
            fig.set_size_inches(5, 4)
            sns.distplot(reconstructed[0, ..., 0].numpy().flatten(), ax=fig.add_subplot(111))
            plt.show()
            fig.clear()

    def _visualize_generation(self, batch=None, epoch=None):
        assert (batch is not None) or (epoch is not None)
        decoder = self.model.get_layer("decoder")
        generated = decoder(self.random_vectors)
        fig = plt.figure(self.fig.number)
        fig.set_size_inches(5, 4)
        for i, gen in enumerate(generated):
            gen = gen[None, :]
            title = f"gen_{i + 1}@{'epoch' if epoch else 'batch'}_{epoch or batch:06}"
            beatbrain.display.show_spec(
                utils.denormalize_spectrogram(gen[0, ..., 0].numpy()),
                title=title,
                cmap=self.cmap,
            )
            fig.tight_layout()
            fig.savefig(self.gen_png / f"{title}.jpg")
            utils.save_image(gen[0, ..., 0], self.gen_raw / f"{title}.exr")
            fig.clear()

    def on_epoch_begin(self, epoch, logs=None):
        if self.frequency == "epoch":
            self._visualize_reconstruction(epoch=epoch)
            self._visualize_generation(epoch=epoch)

    def on_train_batch_begin(self, batch, logs=None):
        if isinstance(self.frequency, int) and (self.total_batch % self.frequency == 0):
            self._visualize_reconstruction(batch=self.total_batch)
            self._visualize_generation(batch=self.total_batch)

    def on_train_batch_end(self, batch, logs=None):
        self.total_batch += 1

In [ ]:
# Hyperparameters
LATENT_DIM = 64
EPOCHS = 200
MODULE_REPEAT = 4
USE_INCEPTION = False
LEARNING_RATE = 1e-4

# Outputs
MODEL_NAME = "rvae_dense_1sample"
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(exist_ok=True, parents=True)
LOG_DIR = Path("../logs") / MODEL_NAME
LOG_DIR.mkdir(exist_ok=True, parents=True)
LOG_FREQUENCY = 100

In [ ]:
tensorboard = TensorBoard(
    log_dir=LOG_DIR,
    update_freq=LOG_FREQUENCY,
    histogram_freq=1,
)
reduce_lr = ReduceLROnPlateau(patience=2, factor=0.1, min_lr=1e-7, verbose=1,)
early_stop = EarlyStopping(patience=5, verbose=1,)
model_saver = ModelCheckpoint(
    str(MODEL_DIR / MODEL_NAME), save_best_only=True, verbose=1,
)
visualizer = VisualizeCallback(
    LOG_DIR,
    LATENT_DIM,
    val_dataset,
    frequency=LOG_FREQUENCY,
    distribution=False,
)

## Instantiate and Train Model

### Train on FMA

In [ ]:
model, encoder, decoder = build_cvae(
    LATENT_DIM,
    IMAGE_DIMS,
    repeat=MODULE_REPEAT,
    use_inception=USE_INCEPTION,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
)
model.summary()
encoder.summary()
decoder.summary()
plot_model(model, to_file=str(LOG_DIR / "model.png"), expand_nested=True, show_shapes=True)

In [ ]:
# Train model
model.fit_generator(
    train_dataset.take(1000),
    epochs=EPOCHS,
    callbacks=[tensorboard, model_saver, reduce_lr, visualizer,],
    validation_data=val_dataset.take(1000),
)

In [ ]:
# OR - Overfit model on 1 batch (from the test dataset, to ensure the same sample each time)
single_sample = tf.data.Dataset.from_tensor_slices(list(test_dataset.take(1)))
model.fit_generator(
    single_sample.repeat(LOG_FREQUENCY),
    epochs=EPOCHS,
    callbacks=[tensorboard, VisualizeCallback(LOG_DIR, LATENT_DIM, single_sample, frequency=LOG_FREQUENCY)],
    validation_data=single_sample,
)